<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2023/blob/main/Unidad_3/02_Divide_y_venceras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np

# Ejemplos

## Multiplicación

Mientras estudiaba los números complejos, Gauss se dio cuenta de que el producto de dos números, definido como:

$$
(a+bi)(c+di) = ac - bd + (bc+ad)i
$$

Podía calcularse utilizando 3 multiplicaciones, en vez de las 4 aparentes. Esto se debe a que:

$$
bc+ad = (a+b)(c+d)-ac-bd
$$

---

Para ver para qué nos sirve esto, consideremos a dos enteros $x$ y $y$, ambos representados con $n$ bits. Por conveniencia, asumimos que $n$ es una potencia de 2, aunque el procedimiento para el caso general es muy similar.

Como primer paso, partimos a $x$ a la mitad. Definimos a las dos partes resultantes como $x_L$ (izquierda) y $x_R$ (derecha). Notemos que podemos recuperar a $x$ de estas dos partes de la siguiente manera:

$$
x = x_L \oplus x_R
$$

Donde $\oplus$ es el operador de concatenación, que "pega" a sus dos operandos. Otra manera de obtener $x$ usando solo operaciones aritméticas es de la siguiente forma:

$$
x = 2^{n/2}\cdot x_L+x_R
$$

Por ejemplo, si $x=10110110$, entonces $x_L=1011$ y $x_R=0110$, y:

$$
x=1011\cdot 2^4 + 0110
$$

Hacemos lo mismo para $y$, partiéndola en $y_L$ y $y_R$. Con esto, el producto de $x$ y $y$ puede escribirse como:

$$
x\cdot y = \left(2^{n/2}\cdot x_L+x_R\right)\left(2^{n/2}\cdot y_L+y_R\right)
$$

Esto es equivalente a:

$$
x\cdot y = 2^n x_Ly_L + 2^{n/2}(x_Ly_R+x_Ry_L) + x_Ry_R
$$

Las sumas tienen una complejidad $O(n)$. Por otro lado, las multiplicaciones por potencias de 2 pueden realizarse utilizando *bit shifts*, los cuales igual tienen complejidad $O(n)$. Entonces, las partes importantes son los 4 productos:

$$
x_Ly_L,\quad x_Ly_R,\quad x_Ry_L,\quad x_Ry_R
$$

Cada uno de estos números ($x_L$, $x_R$, etc.), tiene un tamaño ${n}/{2}$. Por lo tanto, si resolvemos esta multiplicación con el mismo algoritmo, obtenemos la siguiente relación de recurrencia:

$$
T(n) = 4T\left(\frac{n}{2}\right) + O(n)
$$

Aplicando el teorema maestro, llegamos a que $T(n)=O\left(n^2\right)$, que es lo mismo que el algoritmo de multiplicación de primaria.

---

Sin embargo, podemos aplicar el truco de Gauss de una manera similar a la del caso complejo, notando que:

$$
\begin{align}
    x_Ly_R + x_Ry_L = (x_L+x_R)(y_L+y_R) - x_Ly_L - x_Ry_R
\end{align}
$$

Entonces, pasamos de 4 multiplicaciones a 3, lo cual significa que la relación de recurrencia es ahora:

$$
T(n) = 3T\left(\frac{n}{2}\right) + O(n)
$$

Usando el teorema maestro, llegamos a que la complejidad es:

$$
T(n) = O\left(n^{\log_23}\right) \approx O\left(n^{1.59}\right)
$$

Una clara mejoría de nuestro primer intento.

## Mediana

Dada una lista de $n$ números $L$, queremos calcular su mediana. La solución más obvia es ordenarla (que toma tiempo $O(n\log n)$), y luego encontrar el elemento de en medio (que toma tiempo $O(n)$). Sin embargo, hay una mejor solución.

Irónicamente, para resolver esto, sirver considerar un problema más general: dada una lista de números, encontrar el $k$-ésimo elemento más pequeño. Algunos casos particulares son:

* $k=1$, en cuyo caso el resultado es el mínimo.
* $k=n$, el máximo.
* $k=\left\lceil\frac{n}{2}\right\rceil$, la mediana.

Ahora, supongamos que para un número arbitrario $v$, partimos la lista en tres sublistas:

* $L_L$, los números más pequeños que $v$.
* $L_v$, los números iguales a $v$.
* $L_R$, los números mayores a $v$.

Por ejemplo, si tenemos $v=5$ y la lista:

$$
L = [2, 36, 5, 21, 8, 13, 11, 20, 5, 4, 1]
$$

Las tres sublistas resultantes son:

$$
L_L = [2, 4, 1],\qquad L_v = [5, 5], \qquad L_R = [36, 21, 8, 13, 11, 20]
$$

Ahora, supongamos que queremos el octavo elemento más pequeño. Con esta información, sabemos que tenemos que tomar el **tercer** elemento de la lista $L_R$, ya que $|L_L| + |L_v| = 5$. En otras palabras, si tenemos una función $f(L, k)$ que produce el resultado deseado, tenemos que:

$$
f(L, 8) = f(L_R, 3)
$$

En general, la relación de recurrencia para un $k$ arbitrario es:

$$
f = 
\begin{cases}
f(L_L, k) & \text{si}\ k\leq |L_L| \\
v & \text{si}\ |L_L| < k \leq |L_L| + |L_v| \\
f(L_R, k - |L_L| - |L_v|) & \text{si}\ k > |L_L| + |L_v|
\end{cases}
$$

Las sublistas $L_L, L_v, L_R$ pueden calcularse en tiempo lineal. Sin embargo, falta decidir el valor de $v$. Idealmente, nos gustaría que fuese tal que:

$$
|L_L| \approx |L_r| \approx \frac{|L|}{2}
$$

Si esto pasase, entonces el tiempo de ejecución cumpliría la relación:

$$
T(n) = T\left(\frac{n}{2}\right) + O(n)
$$

Sin embargo, esto requiere que $v$ sea la mediana, i.e., nuestro objetivo. Entonces, seguimos una alternativa sencilla: escogemos $v$ *aleatoriamente*.

---

Los casos extremos de esta estrategia son:

**Peor caso** 

Escogemos el número más grande (o el más pequeño) en cada iteración. Entonces nuestra lista solo se encoge por un elemento cada vez. Por lo tanto, el número de iteraciones es del orden:

$$
n + (n-1) + (n-2) + \cdots 1 \sim O(n^2)
$$

**Mejor caso** 

Escogemos la mediana en cada iteración. Por lo tanto, el número de iteraciones es del orden $O(n)$.

Sin embargo, es muy poco probable que cualquiera de estos dos casos ocurra. 

---

En general, decimos que una elección de $v$ es *buena* si cae entre los percentiles 25 y 75 de la lista. Si cumple esta propiedad, entonces $L_L$ y $L_R$ cumplen:

$$
|L_L| \leq \frac{3}{4}|L|,\qquad |L_R|\leq \frac{3}{4}|L|
$$

Dado que 50% de los elementos caen dentro de estos percentiles, ¿cuántas veces, en promedio, tenemos que muestrear $v$ antes de obtener un valor *bueno*? 

Esto es equivalente a preguntar cuántas veces tenemos que tirar una moneda antes de que salga águila. Si en la primera tirada es águila, ya acabamos. Si en la primera25 y 75 tirada es sol, tenemos que repetir. Si $x$ es el número de tiros, de esto se sigue la ecuación:

$$
x = \frac{1}{2}\cdot 1 + \frac{1}{2}(1+x)
$$

Resolviendo, obtenemos $x=2$. 

---

Por lo tanto, regresando a nuestro problema original, esto nos dice que después de dos iteraciones **en promedio**, la lista se habrá encogido a $3/4$ de su tamaño original. Entonces, si $T(n)$ es el tiempo de ejecución **promedio**, tenemos la relación de recurrencia:

$$
T(n) = T\left(\frac{3n}{4}\right) + O(n)
$$

Usando el teorema maestro, se sigue que la complejidad **promedio** es $O(n)$.

---

Finalmente, podemos escribir este algoritmo:

In [8]:
def k_smallest(L, k):
    v = np.random.choice(L)
    LL, Lv, LR = [], [], []
    for x in L:
        if x < v:
            LL.append(x)
        elif x == v:
            Lv.append(x)
        else:
            LR.append(x)

    if k <= len(LL):
        return k_smallest(LL, k)
    elif len(LL) < k <= len(LL) + len(Lv):
        return v
    else:
        return k_smallest(LR, k - len(LL) - len(Lv))

In [14]:
L = np.random.randint(0, 21, 15)
print(list(L))
k_smallest(L, len(L)//2)

[6, 1, 14, 5, 7, 20, 13, 19, 10, 6, 11, 15, 3, 0, 13]


7

# Ejercicios

## Ejercicio 1

Dado un arreglo $L$, para índices $i, j$ con $i<j$, decimos que $L_i$ y $L_j$ forman una *inversión* si $L_i>L_j$. Por ejemplo, el arreglo:

$$
L = [2, 4, 1, 3, 5]
$$

Tiene tres inversiones: $(2,1),(4,1),(4,3)$.

Diseña un algoritmo de divide y vencerás que cuente el número de inversiones en un arreglo. Este debe de correr en tiempo $O(n\log n)$.

*Aquí va la explicación de tu algoritmo, con la demostración de que corre en el tiempo pedido*

In [2]:
# Aquí va tu código

def count_inversions(L):
    pass

## Ejercicio 2

Supón que tienes un conjunto $S$ de $n$ puntos en el plano. Diseña un algoritmo de divide y vencerás para encontrar el par de puntos más cercano. Tu algoritmo debe de correr en tiempo $O(n\log n)$.

*Aquí va la explicación de tu algoritmo, con la demostración de que corre en el tiempo pedido*

In [4]:
# Aquí va tu código

def closest_pair(S):
    pass

# Ejercicio 3

Diseña un algoritmo de divide y vencerás que invierta un arreglo. Por ejemplo, $[1,2,3,4]$ se vuelve $[4,3,2,1]$.

*Aquí va la explicación de tu algoritmo, junto con el análisis de complejidad*

In [5]:
# Aquí va tu código

def reverse_array(A):
    pass